# Lab2
## Overview
1. ศึกษาการจำลองการเคลื่อนที่หุ่นยนต์ผ่านโปรแกรม Simulation (Gazebo)
2. ศึกษาการเชื่อมต่อ Sensor กับ ROS2
3. ศึกษาการสร้าง Algorithm สำหรับควบคุมตำแหน่งปลายมือของหุ่นยนต์

โดยเราจะแบ่งการทำงานออกเป็น 6 ส่วน ได้แก่

## 1. การเชื่อมต่อ IMU เข้ากับ Library XICRO 
ในส่วนนี้เราได้ใช้ node generate_xicro_node.py ของ xicro_pkg ในการสร้าง node สำหรับการเชื่อมต่อ IMU กับ ROS2 ในส่วนของคอมพิวเตอร์ 
และใช้ generate_library.py ของ xicro_pkg ในการสร้าง library สำหรับการเชื่อมต่อ IMU กับ ROS2 ในส่วนของ microcontroller และตัวบอร์ดจะติดต่อกันผ่าน serial port โดยใช้ baudrate 57600 โดยมีการตั้งตัว timer ให้กับการอ่านค่า IMU 10 ครั้งต่อวินาที และส่งค่าขึ้นมาบน ros ผ่าน library ที่สร้างขึ้นมา โค้ดจะอยู่ในไฟล์ Xicro/Demo_IMU/readImu.ino

## 2. ทำโหนด Calibration ของ IMU
เราได้เขียน algorithm สำหรับการ calibration ของ IMU ให้อยู่บน microcontroller โดยใช้ library ที่สร้างขึ้นมาในส่วนที่ 1 โดยการ calibration จะทำงานเมื่อ microcontroller เปิดการทำงาน โดยจะเก็บค่า 100 หรือประมาณ 10 วินาที และทำการคำนวณค่า offset ของแต่ละแกน และนำค่า offset นี้ไปมาลบกับค่าที่อ่านได้จาก IMU ในการทำงานต่อไป และหาความแปรปรวนของแต่ละแกน และส่งขึ้นมาบน ros2

โดยที่ใน high level จะมี package ที่ชื่อว่า imu_complementary_filter ซึ่งจะมี node ที่ชื่อว่า complementary_filter ซึ่งจะทำการรับค่าจาก IMU และทำการ filter และ estimate
<p align="center">
<img src="pic/pic1.png" width="40%">
</p>

<p align="center">
ภาพที่ 1 ภาพตัวอย่างการเชื่อมต่อ IMU กับ ROS2 ผ่าน serial port
</p>

<p align="center">
<img src="pic/pic2.png" width="40%">
</p>

<p align="center">
ภาพที่ 2 ภาพ Topic echo ของ IMU ที่ติดต่อกับ ROS2
</p>

## 3. Algorithm X สำหรับการสั่งงานหุ่นยนต์
โดยเราจะกำหนดเงื่อนไขไว้ว่าหาก 
```
ImuFiltered.linear_acceration.x > threshold : command = commy.backward
ImuFiltered.linear_acceration.x < -threshold : command = commy.forward
ImuFiltered.angular_vocity.y > threshold : command = commy.down
ImuFiltered.angular_vocity.y < -threshold : command = commy.up
ImuFiltered.angular_vocity.z > threshold : command = commy.rotationccw
ImuFiltered.angular_vocity.z < -threshold : command = commy.rotationcw
```
แต่ล่ะคำสั่งนั้นจะถูกนำไปเปลี่ยนพิกัดใน cylindrical coordinates เช่นเดินหน้าจะเปลี่ยนพิกัดในแกน r ขึ้น หมุนเข็มนาฬิกาจะเปลี่ยนพิกัดในแกน theta ขึ้น และเลื่อนขึ้นจะเปลี่ยนพิกัดในแกน z ขึ้น
โดยตัวโค้ดจะอยู่ใน function ชื่อว่า filtered_callback ของไฟล์ lab3/lab3_control/scripts/lab3_Control.py

## 4. การนำไฟล์ URDF/XACRO ไปยังโปรแกรม GAZEBO
เราได้ทำการสร้าง URDF.xacro ของหุ่นยนต์ขึ้นมา ชื่อไฟล์ lab3/lab3_control/description/example_robot.urdf.xacro และนำไปใช้ในการสร้างโปรแกรม GAZEBO ผ่านตัว lauch file lab3/lab3_control/launch/dummy_gazebo.launch.py โดยผลลัพธ์ที่ได้จะเป็น
<p align="center">
<img src="pic/pic3.png" width="40%">
</p>

<p align="center">
ภาพที่ 3 ภาพโมเดล gazebo ของหุ่นยนต์
</p>

## 5. การติดตั้ง ROS2_CONTROLLER ไปยัง GAZEBO
ในไฟล์ lab3/lab3_control/launch/dummy_gazebo.launch.py จะมีการติดตั้ง ros2_controller ไปยัง gazebo โดยใช้ node ros2_control_node ของ package controller_manager โดยมี parameter ที่สำคัญโดยดึงจากไฟล์ lab3/lab3_control/config/myrobot_controllers.yaml และไฟล์ lab3/lab3_control/description/example_gazebo.xacro

## 6. การออกแบบโครงสร้าง Package ใน ROS2
ผลลัพธ์ที่ได้จะเป็น
<p align="center">
<img src="pic/pic4.png" width="40%">
</p>

<p align="center">
ภาพที่ 4 ภาพโครงสร้าง package ของ ROS2 ส่วนที่ผู้จัดทำทำการเพิ่มเข้ามาจากตัวอย่างที่ให้มา
</p>
